# Using `toshiba` Module

In [ ]:
import os
import sys
import glob
import time
import numpy as np
import scipy
import scipy.interpolate
import matplotlib
import matplotlib.pyplot as plt

# Local modules
import toshiba

# iRadar
sys.path.append(os.path.expanduser('~/Developer/iradar/python'))
import iradar

In [ ]:
files = glob.glob(os.path.expanduser('~/Downloads/toshiba/*.nc'))
filename = files[0]

# azimuth_count = 360
range_count = 200

In [ ]:
sweep = toshiba.read(filename, maxgate=range_count)

In [ ]:
# %matplotlib inline
# plt.rcParams['axes.facecolor'] = 'black'
zmap = iradar.colormap.zmap()
cmap = matplotlib.colors.LinearSegmentedColormap.from_list('colors', zmap[:, :3], N=len(zmap))

In [ ]:
plt.figure(dpi=200)
plt.pcolormesh(sweep['z'], cmap=cmap)
plt.clim((-32, 96))
plt.colorbar()
plt.grid()

In [ ]:
r = 1.0e-3 * sweep['rr']
a = np.pi / 180.0 * sweep['aa']
rr, aa = np.meshgrid(r, a)
xx = rr * np.sin(aa)
yy = rr * np.cos(aa)

In [ ]:
import importlib
import cinterp

In [ ]:
cinterp.data = importlib.reload(cinterp.data)

turbines = cinterp.data.toshiba_cells

a_turb = turbines[:, 0] / 180.0 * np.pi
r_turb = turbines[:, 1]
x_turb = r_turb * np.sin(a_turb)
y_turb = r_turb * np.cos(a_turb)

plt.figure(figsize=(5.25, 4), dpi=200)
plt.pcolormesh(xx, yy, sweep['z'], cmap=cmap)
plt.plot(x_turb, y_turb, 'xw')
plt.clim((-32, 96))
plt.colorbar()
plt.grid()
# plt.xlim((-5, 5))
# plt.ylim((-14, -4))
plt.xlim((0, 5))
plt.ylim((-10.5, -5.5))
plt.show()

In [ ]:
def cell2idx(cells, aa, rr):
    